In [62]:
import os
from random import randint, uniform
import re
import numpy as np
import wave
import contextlib


## Data parsing

In [63]:
result = []
n=10000

with open("./wiki_00","r") as f:
    for line in f.readlines():
        line = re.sub("[A-Za-z0-9,.\(\)\"\":;]","",line)
        line=line.split()
        i=0
        if n<=0:
            break
        p=uniform(0,1)
        if p>0.1:
            continue
        while i<len(line) and n>0:
            window = randint(10,20)
            sentence = line[i:i+window]
            i+=window
            if len(sentence)<5:
                continue
            result.append(" ".join(sentence))
            n-=1
            
            

FileNotFoundError: [Errno 2] No such file or directory: './wiki_00'

In [87]:
with open("abubakr_text.txt","w") as out:
    out.write(" ## si \n".join(result)+" ## si")

## Calculating recordings lengths

In [3]:

total = 0
corrupted=0
files=0
for directory in os.listdir("./recordings/"):
    if os.path.isdir("./recordings/"+directory):
        for file in os.listdir("./recordings/"+directory):
            if file.endswith(".wav"):
                files+=1
                fname = "./recordings/"+directory+"/"+file
                try:
                    with contextlib.closing(wave.open(fname,'r')) as f:
                        frames = f.getnframes()
                        rate = f.getframerate()
                        duration = frames / float(rate)
                        total+=duration
                except Exception as e:
                    corrupted+=1

print("total is ",total," s",", ",total/(60*60)," h")

total is  4979.588625000001  s ,  1.3832190625000003  h


In [4]:
print(files)
print(corrupted)

548
79


# Organizing data

### spliting records to train, val and test sets

In [5]:
!mkdir data
!mkdir data/records

In [6]:
# Extracting all the non-corrupted files
wav_files = []
linker = []
for directory in os.listdir("./recordings/"): # parent directory of the recordings, it should contain folders that contain wav,json and txt files
    if os.path.isdir("./recordings/"+directory):
        to_remove = []
        for file in os.listdir("./recordings/"+directory):
            if file.endswith(".txt"):
                linker_data = open("./recordings/"+directory+"/"+file).readlines()
            if file.endswith(".wav"):
                fname = "./recordings/"+directory+"/"+file
                try:
                    with contextlib.closing(wave.open(fname,'r')) as f:
                        frames = f.getnframes()
                        rate = f.getframerate()
                        duration = frames / float(rate)
                        wav_files.append(fname)
                except Exception as e:
                    to_remove.append(file)
        for file in to_remove:
            i=0
            while i<len(linker_data):
                if file in linker_data[i]:
                    linker_data.pop(i)
                i+=1
        linker.extend(linker_data)
        

            

In [7]:
to_copy = " ".join(wav_files)
!cp -t data/records/ {to_copy}

In [8]:
for i,link in enumerate(linker):
    line = link.split(";")[0].split("(")[1].split(")")[0].strip()
    wav = link.split(";")[1].strip().split("/")[-1]
    linker[i] = wav+":"+line


In [9]:
out_linker = open("./data/linker.txt","w")
out_linker.write("\n".join(linker))
out_linker.close()

In [10]:
!mkdir ./data/records/train
!mkdir ./data/records/test
!mkdir ./data/records/val


In [11]:
import numpy as np
np.random.seed(0)
indices = np.random.permutation(len(wav_files))

test_idx = indices[:len(indices)//2]
validation_portion = int(len(indices)//2 * 0.2)
train_idx = indices[len(indices)//2:-validation_portion]
valid_idx = indices[-validation_portion:]

In [12]:
train_set_files = [wav_files[i].split("/")[-1] for i in train_idx]
valid_set_files = [wav_files[i].split("/")[-1] for i in valid_idx]
test_set_files = [wav_files[i].split("/")[-1] for i in test_idx]

In [13]:
to_copy_train = "./data/records/" + " ./data/records/".join(train_set_files)
to_copy_valid = "./data/records/" + " ./data/records/".join(valid_set_files)
to_copy_test = "./data/records/" + " ./data/records/".join(test_set_files)

In [14]:
!mv -t data/records/train/ {to_copy_train}
!mv -t data/records/val/ {to_copy_valid}
!mv -t data/records/test/ {to_copy_test}

### making of chars.txt file

In [51]:
import re
text_file = open("./text.txt","r").readlines() # Modify this to the directory of your txt file that you recorded with
text_data = []
chars = {" ":1,"ε":0}
char_idx = 2
for link in linker:
    file,idx = link.split(":")[0],int(link.split(":")[1].split(" ")[1])-1
    line = text_file[idx]
    line = line.split("##")[0].strip()
    line = re.sub("[\[\]|٪%«»_ـ]","",line) # Clean unnecessary characters from the data, this is for arabic
    line = re.sub("[,،;؛μ\\\/≈٠١٢٣٤٥٦٧٨٩τα'ίηο–ēιäɛθκ\(\)?”“:’.-]","",line) # Clean unnecessary characters from the data, this is for arabic
    line = re.sub('["]',"",line) # Clean unnecessary characters from the data, this is for arabic
    text_data.append((line,file.split(".")[0]))
    char_set = set(line)
    for c in char_set:
        if c not in chars:
            chars[c]=char_idx
            char_idx+=1
            
    

In [52]:
raw_text = "\n".join([wav+":"+line for line,wav in text_data])
with open("data/raw_text_file.txt","w") as f:
    f.write(raw_text)


In [53]:
indices_text = []
for line,wav in text_data:
    line = list(line)
    indicies = []
    for c in line:
        indicies.append(str(chars[c]))
    indices_text.append(wav+" "+" ".join(indicies))

    

In [58]:
indicies_text = "\n".join(indices_text)
with open("data/chars.txt","w") as f:
    f.write(indicies_text)

In [59]:
with open("data/charset.json","w") as js:
    js.write(str(chars))

In [60]:
with open("data/charset.json") as js:
    charset = eval(js.read())

In [61]:
chars

{' ': 1,
 'ε': 0,
 'e': 2,
 'k': 3,
 'i': 4,
 'w': 5,
 'z': 6,
 'H': 7,
 'h': 8,
 's': 9,
 'a': 10,
 'y': 11,
 'd': 12,
 't': 13,
 'ƙ': 14,
 'u': 15,
 'm': 16,
 'b': 17,
 'n': 18,
 'r': 19,
 'l': 20,
 'W': 21,
 'S': 22,
 'K': 23,
 'R': 24,
 'g': 25,
 'F': 26,
 'j': 27,
 'o': 28,
 'f': 29,
 'A': 30,
 'ɓ': 31,
 'T': 32,
 'D': 33,
 'Z': 34,
 'J': 35,
 'c': 36,
 'M': 37,
 'N': 38,
 'B': 39,
 '8': 40,
 '7': 41,
 '1': 42,
 'Ɗ': 43,
 '0': 44,
 '9': 45,
 'ɗ': 46,
 'Y': 47,
 'I': 48,
 '‘': 49,
 'U': 50,
 '4': 51,
 '6': 52,
 'G': 53,
 '5': 54,
 '2': 55,
 'Ƙ': 56,
 '3': 57,
 'L': 58,
 'C': 59,
 'E': 60,
 'P': 61,
 'v': 62,
 'p': 63,
 'q': 64,
 'O': 65}